In [25]:
import numpy as np
import regex as re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import string
import nltk as nlp
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec, Phrases
#from gensim.test.utils import common_texts
#from collections import Counter #like map but worse cuz it senses only the tally --> not for computation :(
import gc
from tensorflow.python.client import device_lib
import tensorflow as tf

In [26]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15979342631785364082
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2907098318
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13215436585619913671
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [27]:
print(tf.test.is_built_with_cuda())

True


#### LOADING IMDB DATASET

In [28]:
dataframe = pd.read_csv(r'..\\IMDB Dataset.csv')

In [29]:
dataframe.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [30]:
dataframe.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [31]:
np.shape(dataframe)

(50000, 2)

In [32]:
dataframe['sentiment'] = dataframe['sentiment'].replace('positive', 1)
dataframe['sentiment'] = dataframe['sentiment'].replace('negative', 0)
dataframe.head()
#in case of non-binary classes it makes more sense to use label encoder rather than replace

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


#### CHECK FOR NULLS AND DUPLICATES

In [33]:
dataframe.isnull().sum()

review       0
sentiment    0
dtype: int64

In [34]:
dataframe.duplicated().sum()

418

In [35]:
dataframe.drop_duplicates(subset='review', keep='first', inplace=True)

In [36]:
np.shape(dataframe)

(49582, 2)

In [37]:
dataframe.duplicated().sum()

0

In [38]:
dataframe.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


#### split into test and train samples

#### REMOVING NON-WORD CHARACTERS FROM THE DATASET

In [39]:
def preprocess(text, pattern):
    if pattern=='[.]+':
        text = re.sub(pattern, '. ', text)
    elif pattern =="[']":
        text =  re.sub(pattern, ' ', text)
    else:
        text = re.sub(pattern, '', text)
    #print(text, '\n')
    return text

In [40]:
dataframe['review'] = np.vectorize(preprocess)(dataframe['review'], '<[^>]*>') #remove markup
#print(dataframe.loc[0, 'review'])
dataframe.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [41]:
dataframe['review'] = np.vectorize(preprocess)(dataframe['review'], '[.]+') #remove ... and replace with .
#print(dataframe.loc[0, 'review'])
dataframe.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming te...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [42]:
dataframe['review'] = np.vectorize(preprocess)(dataframe['review'], '[0-9]+') #remove ... and replace with .
#print(dataframe.loc[0, 'review'])
dataframe.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming te...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [43]:
dataframe['review'] = np.vectorize(preprocess)(dataframe['review'], "[']") #remove ... and replace with .
#print(dataframe.loc[0, 'review'])
dataframe.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming te...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there s a family where a little boy ...,0
4,"Petter Mattei s ""Love in the Time of Money"" is...",1


In [44]:
dataframe['review'] = np.vectorize(preprocess)(dataframe['review'], '[^\w\s\']*') #remove everything that's not word space or '
# ' is left to handle contractions
#print(dataframe.loc[0, 'review'])
dataframe.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there s a family where a little boy ...,0
4,Petter Mattei s Love in the Time of Money is a...,1


In [45]:
dataframe['review'] = dataframe['review'].str.lower()#make it lower
#print(dataframe.loc[0, 'review'])
dataframe.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there s a family where a little boy ...,0
4,petter mattei s love in the time of money is a...,1


In [46]:
print(dataframe.loc[0, 'review'])

one of the other reviewers has mentioned that after watching just  oz episode you ll be hooked  they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go  trust me this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs sex or violence  its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda  em city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldn t dare  forget p

#### DOES IT MAKE SENSE TO REMOVE SOME WORDS TO REDUCE COMPUTATION?

In [47]:
count = CountVectorizer()
bag = count.fit_transform(dataframe['review'])
len(count.vocabulary_)

148992

#### Got over 2 Lakh words --> it makes sense to remove some words like articles and prepositions out
#### Better to remove stop words first (Why? --> documentation wip)

In [48]:
#something with tfidf
#question: does it make sense to do tfidf first and then remove stop words using the nltk corpus or 
#remove stop words using the corpus first then perform tfidf next

In [49]:
nlp.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rps24\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
stop = set(stopwords.words('english')) #set makes serach O(1)
#originally stopwords.words('english') yields a list
print(stop)

{'were', 've', 'll', 'with', 'himself', "weren't", 'weren', 'why', 'very', 'the', 'once', 'aren', "haven't", 'does', 'those', 'but', 'to', 'on', 'itself', 'yourself', "didn't", 'our', 'from', 'more', 'i', 'so', "shan't", 'ourselves', 'as', 'didn', 'wouldn', 're', "hadn't", 'd', 'any', 'its', 'theirs', 'only', 'under', 'this', 'don', 'at', 'shan', 'before', 'up', 'm', "doesn't", 'some', 'few', 'y', 'off', 'him', 'against', 'through', 'no', 'doesn', 'both', 'your', 'yourselves', 'you', "you've", "won't", 'until', 'yours', 'if', 'their', "should've", 'what', 'his', 'hadn', 'wasn', 'that', "don't", 'myself', 'can', "shouldn't", "you'd", 'couldn', 'ain', 'she', 'was', 'because', 'mightn', 'whom', 'be', 'which', "needn't", 'and', 'her', 'about', 'doing', 'too', 'am', 'each', 'when', "you'll", 'not', "mustn't", "hasn't", 'other', 'have', 'out', 'by', 'it', 'should', 'between', "mightn't", 'these', 'now', 'he', 'themselves', 'been', "that'll", 'who', 'had', "it's", 'down', 'hers', 'for', 'havi

In [51]:
ps  = PorterStemmer()

In [52]:
def remove_stopwords_and_stem(text):
    text_ = word_tokenize(text)
    tokens = []
    #print(text_)
    for word in text_:
        if word not in stop:
            tokens.append(ps.stem(word))
    #return lemmatization(tokens)
    #print(tokens)
    text = ' '.join(tokens) #send only tokens sent as a joined sentence
    return text 

In [53]:
dataframe['review'] = np.vectorize(remove_stopwords_and_stem)(dataframe['review'])
print(dataframe.loc[0, 'review'])

one review mention watch oz episod hook right exactli happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch darker side


In [54]:
lemmatizer = WordNetLemmatizer()
nlp.download('wordnet')
reviews = []

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rps24\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [55]:
def lemmatization(text):
    text_ = word_tokenize(text)
    tokens = []
    for word in text_:
        tokens.append(lemmatizer.lemmatize(word))
    text = ' '.join(tokens)
    reviews.append(tokens)
    return text

In [56]:
dataframe['review'] = np.vectorize(lemmatization)(dataframe['review'])
print(dataframe.loc[0, 'review'])

one review mention watch oz episod hook right exactli happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch darker side


#### word2vec

In [57]:
all_reviews = np.array(reviews)
#del reviews

C:\Users\rps24\AppData\Local\Temp/ipykernel_18480/4239875209.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_reviews = np.array(reviews)


In [160]:
gc.collect()

8

In [ ]:
word2vec_model = Word2Vec(reviews, window = 3, min_count = 1, sg = 2, size = 256, workers = 5) #sg --> skipgram
#workers --> number of threads in use
#need to save model here

In [ ]:
word2vec_model.wv.similarity('saw', 'may')

In [ ]:
word2vec_model.wv.similarity('saw', 'say')

In [ ]:
word2vec_model.wv.similarity('say', 'may')

In [ ]:
word2vec_model.wv.similarity('gangsta', 'latino')